In [13]:
import torch
from datasets import load_mnist, get_observation_pixels
import numpy as np
import matplotlib.pyplot as plt

BATCH_SIZE = 128
train_loader, test_loader, val_loader = load_mnist(BATCH_SIZE)

In [14]:
import pytorch_lightning as pl
from models import VAE

class VAEModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.VAE = VAE()
    
    def forward(self, x, x_cond):
        return self.VAE(x)
    
    def training_step(self, batch, batch_idx):
        x, x_cond, y = batch
        output, z_mean, z_log_var, z = self.VAE(x)
        recon_loss, kl_loss, loss = self.VAE.loss(x, output, z_mean, z_log_var)
        self.log('train_loss', loss)
        self.log('train_recon_loss', recon_loss)
        self.log('train_kl_loss', kl_loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, x_cond, y = batch
        output, z_mean, z_log_var, z = self.VAE(x)
        recon_loss, kl_loss, loss = self.VAE.loss(x, output, z_mean, z_log_var)
        self.log('val_loss', loss)
        self.log('val_recon_loss', recon_loss)
        self.log('val_kl_loss', kl_loss)
        return loss
    
    def test_step(self, batch, batch_idx):
        x, x_cond, y = batch
        output, z_mean, z_log_var, z = self.VAE(x)
        recon_loss, kl_loss, loss = self.VAE.loss(x, output, z_mean, z_log_var)
        self.log('test_loss', loss)
        self.log('test_recon_loss', recon_loss)
        self.log('test_kl_loss', kl_loss)
        return loss
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

In [16]:
from pytorch_lightning.callbacks import RichProgressBar
scvae = VAEModel()
trainer = pl.Trainer(accelerator='gpu', devices=1, max_epochs=40, enable_progress_bar=True, callbacks=[RichProgressBar()])
trainer.fit(scvae, train_loader, val_loader)
#save
trainer.save_checkpoint('vae_40.ckpt')

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━┳━━━━━━┳━━━━━━━━┓
┃   ┃ Name ┃ Type ┃ Params ┃
┡━━━╇━━━━━━╇━━━━━━╇━━━━━━━━┩
│ 0 │ VAE  │ VAE  │  854 K │
└───┴──────┴──────┴────────┘

Trainable params: 854 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 854 K                                                                                                
Total estimated model params size (MB): 3

Output()